## ETL Bank Market Cap
Projek ini melakukan proses ETL(Ektract Transform Load) dengan mencoba menngambil data market cap dari berbagai bank
di seluruh dunia yang diambil dari 2 sumber data yang berbeda, yang kemudian data dari 2 sumber yang berbeda tersebut
akan diekstrak dalam satu file, selanjutnya dilakukan proses transform dengan mengubah nilai marketcap Us ke kurs GBP

In [1]:
# Memanggil beberapa liblary yang dibutuhkan
import glob
import pandas as pd
from datetime import datetime

In [2]:
# Mengambil data dari beberapa sumber
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2022-07-27 11:26:47--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json.2’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2022-07-27 11:26:47 (41.4 MB/s) - ‘bank_market_cap_1.json.2’ saved [2815/2815]

--2022-07-27 11:26:48--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%2

In [3]:
# memeriksa isi data 1
data1 = pd.read_json('bank_market_cap_1.json')
data1.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


In [4]:
# Memeriksa isi data 2
data2 = pd.read_json('bank_market_cap_2.json')
data2.head()

,Name,Market Cap (US$ Billion)
35,Banco Bilbao Vizcaya Argentaria,60.678
36,Japan Post Bank,60.525
37,The Bank of New York Mellon,59.823
38,Shanghai Pudong Development Bank,58.796
39,Industrial Bank (China),57.388


In [5]:
# Melihat jumlah data1
data1.shape

(70, 2)

In [6]:
# Melihat jumlah data2
data2.shape

(35, 2)

In [7]:
# Membuat fungsi ekstrak untuk file json
def ekstrak_json(file_mentah):
    df = pd.read_json(file_mentah)
    return df

In [8]:
# Membuat fungsi ekstrak untuk file csv
def ekstrak_csv(file_mentah):
    df = pd.read_csv(file_mentah, index_col = 0)
    return df

In [9]:
# Mengekstrak seluruh data yang didapat
def ekstrak():
    ekstrak_data = pd.DataFrame(columns=['Name','Market Cap (US$ Billion)'])

        
    for jsonfile in glob.glob("*.json"):
        ekstrak_data = ekstrak_data.append(ekstrak_json(jsonfile), ignore_index=True)
    
    
    return ekstrak_data

In [10]:
# Mencari faktor pengali dari kurs US ke GBP 
exchange_rate = ekstrak_csv('exchange_rates.csv')["Rates"].loc["GBP"]
exchange_rate

0.7323984208000001

In [11]:
#Transform data
def transform(data):
    data["Market Cap (GBP$ Billion)"] = round( data["Market Cap (US$ Billion)"] * exchange_rate)
    
    return data
    

In [12]:
#Loading data
def load(data_to_load):
    data_to_load.to_csv("bank_market_cap_gbp.csv")

In [13]:
#Logging data
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

In [14]:
log("ETL Market Cap dimulai")

In [15]:
# Melihat hasil ekstrak data
log("Extract Market Cap dimulai")
extracted_data = ekstrak()
log("Extract Market Cap diakhiri")
extracted_data

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399
...,...,...
100,Ping An Bank,37.993
101,Standard Chartered,37.319
102,United Overseas Bank,35.128
103,QNB Group,33.560


In [16]:
#Melihat hasil tranform data
log("Transform phase dimulai")
transformed_data = transform(extracted_data)
log("Transform phase diakhiri")
transformed_data 

,Name,Market Cap (US$ Billion),Market Cap (GBP$ Billion)
0,JPMorgan Chase,390.934,286.0
1,Industrial and Commercial Bank of China,345.214,253.0
2,Bank of America,325.331,238.0
3,Wells Fargo,308.013,226.0
4,China Construction Bank,257.399,189.0
...,...,...,...
100,Ping An Bank,37.993,28.0
101,Standard Chartered,37.319,27.0
102,United Overseas Bank,35.128,26.0
103,QNB Group,33.560,25.0


In [17]:
log("Load phase dimulai")
load(transformed_data)
log("Load phase diakhiri")

In [18]:
log("ETL Market Cap diakhiri")